![Verne Academy Summit 2024](https://github.com/javendia/verne-academy-summit-2024/blob/main/header.png?raw=true)

## Paso 1: Ingesta de datos
- Leemos el fichero **DeliveryMethods.csv**, especificando el formato (CSV) e indicando que el archivo contiene las cabeceras de las columnas
- Creamos la tabla delta destino en caso de no existir

In [1]:
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions import when, lit, col

df = spark.read.load('Files/wwi/DeliveryMethods.csv',
    format='csv',
    header=True
)

DeltaTable.createIfNotExists(spark) \
     .tableName("Delivery_Methods") \
     .addColumn("DeliveryMethodID", IntegerType()) \
     .addColumn("DeliveryMethodName", StringType()) \
     .addColumn("LastUpdated", TimestampType()) \
     .execute()

deltaTable = DeltaTable.forPath(spark, 'Tables/delivery_methods')

StatementMeta(, 79281b84-45f2-4a0e-8768-208c7c5c5f55, 3, Finished, Available)

## Paso 2: Transformación
- Seleccionamos las columnas necesarias
- Devolvemos un valor controlado para la columna **DeliveryMethodName** en caso de ser nula o vacía

In [2]:
df = df.select("DeliveryMethodID", "DeliveryMethodName") \
        .withColumn("DeliveryMethodName", when((col("DeliveryMethodName").isNull() | (col("DeliveryMethodName")=="")),lit("N/A")).otherwise(col("DeliveryMethodName")))

StatementMeta(, 79281b84-45f2-4a0e-8768-208c7c5c5f55, 4, Finished, Available)

## Paso 3: Creación de vista temporal
- Creamos la vista temporal **vw_delivery_methods** donde almacenar los datos tratados anteriormente

In [3]:
df.createOrReplaceTempView("vw_delivery_methods")

StatementMeta(, 79281b84-45f2-4a0e-8768-208c7c5c5f55, 5, Finished, Available)

## Paso 4: Instrucción MERGE

- Si el valor del registro para la columna **DeliveryMethodID** existe **(MATCHED)** en la tabla destino y alguna columna difiere del registro existente, actualizamos la fila
- Si el valor del registro para la columna **DeliveryMethodID** no existe **(NOT MATCHED)** en la tabla destino, insertamos una nueva fila
- En caso de que el registro de la tabla destino no exista en el fichero origen, eliminamos esa fila

In [4]:
%%sql

MERGE INTO delivery_methods AS target
USING vw_delivery_methods AS source
ON target.DeliveryMethodID = source.DeliveryMethodID
WHEN MATCHED AND 
(
    target.DeliveryMethodName <> source.DeliveryMethodName
)
THEN
UPDATE SET
    target.DeliveryMethodName = source.DeliveryMethodName
    ,target.LastUpdated = CURRENT_TIMESTAMP()
WHEN NOT MATCHED THEN
INSERT (DeliveryMethodID, DeliveryMethodName, LastUpdated)
VALUES (source.DeliveryMethodID, source.DeliveryMethodName, CURRENT_TIMESTAMP())
WHEN NOT MATCHED BY SOURCE THEN
DELETE;

StatementMeta(, 79281b84-45f2-4a0e-8768-208c7c5c5f55, 6, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

## Paso 5: Eliminación vista temporal

In [5]:
spark.catalog.dropTempView("vw_delivery_methods")

StatementMeta(, 79281b84-45f2-4a0e-8768-208c7c5c5f55, 7, Finished, Available)

True